## Table des Matières

1. [Introduction](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#Introduction)
2. [Étape 1 : Importer les Bibliothèques Nécessaires](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-1-Importer-les-Biblioth%C3%A8ques-N%C3%A9cessaires)
3. [Étape 2 : Définir les Fonctions de Prétraitement et d’Augmentation](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-2-D%C3%A9finir-les-Fonctions-de-Pr%C3%A9traitement-et-d%E2%80%99Augmentation)
4. [Étape 3 : Définir la Fonction de Création des Modèles](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-3-D%C3%A9finir-la-Fonction-de-Cr%C3%A9ation-des-Mod%C3%A8les)
5. [Étape 4 : Définir la Fonction de Téléchargement Audio depuis YouTube](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-4-D%C3%A9finir-la-Fonction-de-T%C3%A9l%C3%A9chargement-Audio-depuis-YouTube)
6. [Étape 5 : Initialiser les Hyperparamètres](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-5-Initialiser-les-Hyperparam%C3%A8tres)
7. [Étape 6 : Charger et Prétraiter les Données](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-6-Charger-et-Pr%C3%A9traiter-les-Donn%C3%A9es)
8. [Étape 7 : Itérer sur les Combinaisons d'Hyperparamètres et Entraîner les Modèles](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-7-It%C3%A9rer-sur-les-Combinaisons-d'Hyperparam%C3%A8tres-et-Entra%C3%AEner-les-Mod%C3%A8les)
9. [Étape 8 : Analyser et Sauvegarder les Résultats](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-8-Analyser-et-Sauvegarder-les-R%C3%A9sultats)
10. [Étape 9 : Visualiser les Meilleurs Résultats](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-9-Visualiser-les-Meilleurs-R%C3%A9sultats)
11. [Étape 10 : Sauvegarder et Charger les Meilleurs Modèles](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-10-Sauvegarder-et-Charger-les-Meilleurs-Mod%C3%A8les)
12. [Étape 11 : Prédiction sur de Nouveaux Échantillons](https://chatgpt.com/c/6755b38f-32a8-8010-b45f-f9ddc4e36e80#%C3%89tape-11-Pr%C3%A9diction-sur-de-Nouveaux-%C3%89chantillons)

## Introduction

Dans ce notebook, nous allons :

1. **Charger et prétraiter les données audio** en extrayant les caractéristiques MFCC avec augmentation de données.
2. **Définir et entraîner différents modèles** (CNN 2D, CNN profond, RNN avec LSTM/GRU) en testant diverses combinaisons d'hyperparamètres tels que la taille du batch, le nombre d'époques, le taux d'apprentissage et le nombre de coefficients MFCC.
3. **Évaluer les performances** de chaque modèle et collecter les métriques dans un tableau pour une analyse comparative.
4. **Visualiser les meilleurs résultats** et **sauvegarder les modèles** optimaux pour une utilisation future.

## Étape 1 : Importer les Bibliothèques Nécessaires

Tout d’abord, importons toutes les bibliothèques nécessaires pour le prétraitement des données, la création des modèles, et l’évaluation des performances.

In [ ]:
import os
import numpy as np
import pandas as pd
import librosa
import joblib
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, LSTM, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import yt_dlp as youtube_dl

# Pour afficher les graphiques directement dans le notebook
%matplotlib inline

## Étape 2 : Définir les Fonctions de Prétraitement et d’Augmentation

Nous allons définir les fonctions nécessaires pour :

- **Augmenter les données audio** (ajout de bruit, modification de la hauteur et de la vitesse).
- **Padder ou tronquer les MFCCs** pour assurer une longueur fixe.
- **Extraire les MFCCs** avec augmentation.

In [ ]:
def augment_audio(audio, sr):
    """
    Applique des augmentations sur l'audio : ajout de bruit, modification de la hauteur et de la vitesse.
    """
    # Ajout de bruit
    noise = np.random.randn(len(audio)) * 0.005
    audio_with_noise = audio + noise

    # Modifier la hauteur en fréquence
    try:
        pitch_factor = np.random.uniform(-5, 5)  # En demi-tons
        audio_pitch_shifted = librosa.effects.pitch_shift(audio, sr=sr, n_steps=pitch_factor)
    except:
        audio_pitch_shifted = audio

    # Modifier la vitesse
    try:
        audio_stretched = librosa.effects.time_stretch(audio, rate=np.random.uniform(0.8, 1.2))
    except:
        audio_stretched = audio

    return [audio_with_noise, audio_pitch_shifted, audio_stretched]

def pad_or_truncate_mfcc(mfccs, fixed_length):
    """
    Ajuste la longueur des MFCCs (n_mfcc, T) à (n_mfcc, fixed_length)
    - Tronque si T > fixed_length
    - Ajoute du padding (zéros) si T < fixed_length
    """
    length = mfccs.shape[1]
    if length > fixed_length:
        mfccs = mfccs[:, :fixed_length]
    elif length < fixed_length:
        pad_width = fixed_length - length
        mfccs = np.pad(mfccs, ((0, 0), (0, pad_width)), mode='constant')
    return mfccs

def extract_features_with_augmentation(file_path, fixed_length, n_mfcc=40):
    """
    Extrait les MFCCs d'un fichier audio avec augmentation de données.
    """
    try:
        audio, sr = librosa.load(file_path, duration=30)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        mfccs = pad_or_truncate_mfcc(mfccs, fixed_length)

        # Augmentations
        augmented_audios = augment_audio(audio, sr)
        augmented_mfccs = []
        for a in augmented_audios:
            m = librosa.feature.mfcc(y=a, sr=sr, n_mfcc=n_mfcc)
            m = pad_or_truncate_mfcc(m, fixed_length)
            augmented_mfccs.append(m)

        # On renvoie la liste (original + augmentations) directement sous forme (n_mfcc, T)
        all_mfccs = [mfccs] + augmented_mfccs
        return all_mfccs
    except Exception as e:
        print(f"Erreur lors du traitement du fichier {file_path}: {e}")
        return []

## Étape 3 : Définir la Fonction de Création des Modèles

Nous allons définir une fonction qui crée différents types de modèles en fonction des hyperparamètres spécifiés.

In [ ]:
def create_model(model_type, input_shape, num_classes, learning_rate=0.001):
    """
    Crée un modèle de deep learning en fonction du type spécifié.
    """
    model = Sequential()

    if model_type == 'CNN_2D_basic':
        # Modèle CNN 2D de base
        model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(128, (3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Flatten())
        model.add(Dense(256, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))

    elif model_type == 'CNN_2D_deep':
        # Modèle CNN 2D plus profond
        model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Conv2D(32, (3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(64, (3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(64, (3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Conv2D(128, (3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(Conv2D(128, (3,3), activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D((2,2)))
        model.add(Dropout(0.3))

        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.4))
        model.add(Dense(num_classes, activation='softmax'))

    elif model_type == 'RNN_LSTM':
        # Modèle avec couches LSTM
        # Reshape pour LSTM : (N, T, n_mfcc)
        model.add(Flatten(input_shape=input_shape))  # Convertir en séquence pour LSTM
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(LSTM(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(LSTM(64))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))

    elif model_type == 'RNN_GRU':
        # Modèle avec couches GRU
        # Reshape pour GRU : (N, T, n_mfcc)
        model.add(Flatten(input_shape=input_shape))  # Convertir en séquence pour GRU
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.3))
        model.add(GRU(128, return_sequences=True))
        model.add(Dropout(0.3))
        model.add(GRU(64))
        model.add(Dropout(0.3))
        model.add(Dense(num_classes, activation='softmax'))

    else:
        raise ValueError(f"Type de modèle inconnu: {model_type}")

    # Compilation du modèle
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

## Étape 4 : Définir la Fonction de Téléchargement Audio depuis YouTube

Cette fonction permet de télécharger un fichier audio depuis une URL YouTube pour tester les prédictions.

In [ ]:
def download_youtube_audio(youtube_url, output_path="downloaded_audio.wav"):
    """
    Télécharge l'audio d'une vidéo YouTube et le sauvegarde en fichier WAV.
    """
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'temp_audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
    }

    # Télécharger l'audio
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    # Renommer le fichier pour l'utiliser
    if os.path.exists("temp_audio.wav"):
        os.rename("temp_audio.wav", output_path)
        print(f"Audio téléchargé et sauvegardé sous : {output_path}")
    else:
        print("Erreur lors du téléchargement.")


## Étape 5 : Initialiser les Hyperparamètres

Définissons les différentes valeurs des hyperparamètres que nous allons tester.

In [ ]:
# Définition des hyperparamètres à tester
batch_sizes = [16, 32, 64]
epochs_list = [20, 50, 100]
learning_rates = [0.001, 0.0001, 0.00001]
model_types = ['CNN_2D_basic', 'CNN_2D_deep', 'RNN_LSTM', 'RNN_GRU']  # Ajouter d'autres modèles si nécessaire
n_mfcc_values = [20, 40, 60]  # "token size" interprété comme nombre de MFCCs

# Préparation du tableau de résultats
results = []

# Créer un répertoire pour sauvegarder les modèles
os.makedirs('models', exist_ok=True)

## Étape 6 : Charger et Prétraiter les Données

Pour chaque valeur de `n_mfcc`, nous chargerons et prétraiterons les données audio avec augmentation.

In [ ]:
def load_audio_features_with_augmentation(data_path, fixed_length, n_mfcc=40):
    """
    Charge les features audio avec augmentation et ajuste les MFCCs à une longueur fixe.
    """
    genres = os.listdir(data_path)
    X_list = []
    y_list = []

    for genre in genres:
        genre_path = os.path.join(data_path, genre)
        for file_name in os.listdir(genre_path):
            file_path = os.path.join(genre_path, file_name)
            mfccs_list = extract_features_with_augmentation(file_path, fixed_length=fixed_length, n_mfcc=n_mfcc)
            for m in mfccs_list:
                # m est de forme (n_mfcc, fixed_length)
                X_list.append(m)
                y_list.append(genre)

    X = np.array(X_list)  # X: (N, n_mfcc, fixed_length)
    y = np.array(y_list)

    # Ajout de la dimension "canal" pour le CNN 2D : (N, n_mfcc, fixed_length, 1)
    X = X[..., np.newaxis]
    return X, y


## Étape 7 : Itérer sur les Combinaisons d'Hyperparamètres et Entraîner les Modèles

Nous allons maintenant itérer sur toutes les combinaisons possibles d'hyperparamètres et de types de modèles, entraîner chaque modèle, évaluer ses performances, et enregistrer les résultats.

In [ ]:
# Chemin vers les données audio
data_path = "Data/genres_original"

# Définir fixed_length en fonction de n_mfcc et des paramètres de vos MFCC
fixed_length = 1293  # À ajuster selon votre dataset (en fonction du sr et hop_length)

# Itération sur les différentes valeurs de n_mfcc (token size)
for n_mfcc in n_mfcc_values:
    print(f"\nTraitement pour n_mfcc = {n_mfcc}")

    # Charger les données avec augmentation
    X, y = load_audio_features_with_augmentation(data_path, fixed_length, n_mfcc=n_mfcc)

    print(f"Shape des données: X={X.shape}, y={y.shape}")

    # Encodage des labels
    encoder = LabelEncoder()
    y_encoded = encoder.fit_transform(y)
    y_categorical = to_categorical(y_encoded, num_classes=len(encoder.classes_))

    # Split des données en ensembles d'entraînement et de test
    X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

    # Normalisation des données
    X_train = X_train / np.max(X_train)
    X_test = X_test / np.max(X_test)

    # Itération sur les différentes combinaisons d'hyperparamètres
    for model_type, batch_size, epochs, lr in itertools.product(model_types, batch_sizes, epochs_list, learning_rates):
        print(f"Entraînement du modèle: {model_type}, Batch Size: {batch_size}, Epochs: {epochs}, Learning Rate: {lr}")

        # Définir le modèle
        input_shape = X_train.shape[1:]  # (n_mfcc, fixed_length, 1)
        num_classes = len(encoder.classes_)
        model = create_model(model_type, input_shape, num_classes, learning_rate=lr)

        # Définir le callback EarlyStopping
        early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Optionnel : Définir un ModelCheckpoint pour sauvegarder le meilleur modèle
        model_save_path = f"models/n_mfcc_{n_mfcc}/{model_type}_bs{batch_size}_ep{epochs}_lr{lr}.h5"
        os.makedirs(os.path.dirname(model_save_path), exist_ok=True)
        checkpoint = ModelCheckpoint(model_save_path, monitor='val_accuracy', save_best_only=True, verbose=0)

        # Entraîner le modèle
        history = model.fit(X_train, y_train, validation_split=0.2, epochs=epochs,
                            batch_size=batch_size, callbacks=[early_stopping, checkpoint], verbose=0)  # verbose=0 pour moins de sorties

        # Évaluer le modèle
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        val_loss = min(history.history['val_loss'])
        val_accuracy = max(history.history['val_accuracy'])

        # Enregistrer les résultats
        result = {
            'n_mfcc': n_mfcc,
            'model_type': model_type,
            'batch_size': batch_size,
            'epochs': epochs,
            'learning_rate': lr,
            'accuracy': accuracy,
            'loss': loss,
            'validation_accuracy': val_accuracy,
            'validation_loss': val_loss
        }
        results.append(result)
        print(f"Résultat: {result}")

**Remarques :**

- **Boucles imbriquées** : Nous itérons d'abord sur les différentes valeurs de `n_mfcc`, puis sur toutes les combinaisons d'hyperparamètres pour chaque type de modèle.
- **Sauvegarde des Modèles** : Utilisation de `ModelCheckpoint` pour sauvegarder le meilleur modèle basé sur la précision de validation (`val_accuracy`) pour chaque combinaison d'hyperparamètres.
- **Normalisation** : Les MFCCs sont normalisés en les divisant par leur maximum pour faciliter la convergence du modèle.

## Étape 8 : Analyser et Sauvegarder les Résultats

Après avoir entraîné tous les modèles, nous allons collecter les résultats dans un tableau et les sauvegarder dans un fichier CSV pour une analyse ultérieure.

In [ ]:
# Convertir les résultats en DataFrame et sauvegarder
results_df = pd.DataFrame(results)
results_df.to_csv('model_results.csv', index=False)
print("\nTous les résultats ont été sauvegardés dans 'model_results.csv'.")


## Étape 9 : Visualiser les Meilleurs Résultats

Nous allons afficher les 10 meilleures configurations basées sur la précision de test.

In [ ]:
# Afficher les meilleurs résultats
best_results = results_df.sort_values(by='accuracy', ascending=False).head(10)
print("\nTop 10 des meilleures configurations :")
display(best_results)

### Visualisation des Résultats

Pour mieux comprendre les performances des différents modèles et hyperparamètres, nous pouvons créer des visualisations.

In [ ]:
# Heatmap de la précision en fonction du modèle et du taux d'apprentissage pour chaque n_mfcc
for n_mfcc in n_mfcc_values:
    plt.figure(figsize=(12, 8))
    subset = results_df[results_df['n_mfcc'] == n_mfcc]
    pivot_table = subset.pivot_table(values='accuracy', index='model_type', columns='learning_rate', aggfunc='mean')
    sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
    plt.title(f"Précision moyenne par modèle et taux d'apprentissage (n_mfcc={n_mfcc})")
    plt.ylabel("Type de Modèle")
    plt.xlabel("Taux d'Apprentissage")
    plt.show()


In [ ]:
# Boxplot de la précision par type de modèle
plt.figure(figsize=(14, 8))
sns.boxplot(x='model_type', y='accuracy', data=results_df)
plt.title("Distribution de la Précision par Type de Modèle")
plt.xlabel("Type de Modèle")
plt.ylabel("Précision")
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Scatter plot de la précision en fonction du taux d'apprentissage pour chaque modèle
plt.figure(figsize=(14, 8))
sns.scatterplot(x='learning_rate', y='accuracy', hue='model_type', style='n_mfcc', data=results_df)
plt.xscale('log')
plt.title("Précision en fonction du Taux d'Apprentissage par Type de Modèle")
plt.xlabel("Taux d'Apprentissage (log scale)")
plt.ylabel("Précision")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()


## Étape 10 : Sauvegarder et Charger les Meilleurs Modèles

Pour réutiliser les meilleurs modèles sans avoir à les réentraîner, nous pouvons les sauvegarder et les charger ultérieurement.

### Sauvegarde des Meilleurs Modèles

Les modèles sont déjà sauvegardés pendant l'entraînement grâce au callback `ModelCheckpoint`. Assurez-vous que les chemins des modèles sont correctement définis dans les résultats.

### Chargement d'un Modèle Sauvegardé

Voici comment charger un modèle sauvegardé et l'utiliser pour faire des prédictions.

In [ ]:
# Exemple de chargement d'un modèle sauvegardé
best_model_path = 'models/n_mfcc_40/CNN_2D_deep_bs32_ep50_lr0.001.h5'  # Remplacez par le chemin de votre meilleur modèle
best_model = load_model(best_model_path)

# Charger l'encodeur des labels (si non déjà chargé)
encoder = joblib.load('label_encoder.joblib')

# Fonction de prédiction
def predict_genre(file_path, model, encoder, fixed_length=1293, n_mfcc=40):
    """
    Prédit le genre musical d'un fichier audio.
    """
    try:
        audio, sr = librosa.load(file_path, duration=30)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=n_mfcc)
        mfccs = pad_or_truncate_mfcc(mfccs, fixed_length)
        mfccs = mfccs[..., np.newaxis]  # Ajouter la dimension canal
        mfccs = np.expand_dims(mfccs, axis=0)  # Ajouter la dimension batch

        # Normalisation
        mfccs = mfccs / np.max(mfccs)

        # Prédiction
        predicted_genre_index = np.argmax(model.predict(mfccs), axis=-1)
        predicted_genre = encoder.inverse_transform(predicted_genre_index)[0]
        return predicted_genre
    except Exception as e:
        print(f"Erreur lors de la prédiction : {e}")
        return None